In [383]:
# dataframe and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [472]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')

variables=pd.read_csv('VariableDefinitions.csv')
ss=pd.read_csv('SampleSubmission.csv')

In [473]:
train.head()


,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [474]:
train['bank_account'] =train['bank_account'].map({'Yes': 1, 'No': 0})

In [475]:
train['bank_account']

0        1
1        0
2        1
3        0
4        0
        ..
23519    0
23520    0
23521    0
23522    0
23523    0
Name: bank_account, Length: 23524, dtype: int64

In [476]:
train.head()

,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,1,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,0,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,1,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,0,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,0,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [510]:
X_train = train.drop(['year', 'uniqueid', 'bank_account', 'country'], axis=1)
y_train = train['bank_account']
X_test = test.drop(['year', 'uniqueid', 'country'], axis=1)
#import preprocessing module
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

# Cobvert target label to numerical Data
le = LabelEncoder()

In [508]:
# function to preprocess our data from train models
def preprocessing_data(data):

    # Convert the following numerical labels from interger to float
    float_array = data[["household_size", "age_of_respondent"]].values.astype(float)
    
    # categorical features to be onverted to One Hot Encoding
    categ = ["relationship_with_head",
             "marital_status",
             "education_level",
             "job_type",
             ]
    
    # One Hot Encoding conversion
    data = pd.get_dummies(data, prefix_sep="_", columns=categ)
    
    # Label Encoder conversion
    data["location_type"] = le.fit_transform(data["location_type"])
    data["cellphone_access"] = le.fit_transform(data["cellphone_access"])
    data["gender_of_respondent"] = le.fit_transform(data["gender_of_respondent"])
    
  
    
    # scale our data into range of 0 and 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data)
    
    return data   

In [511]:
processed_train = preprocessing_data(X_train)
processed_test = preprocessing_data(X_test)

In [512]:
from sklearn import svm

X_Train, X_Val, y_Train, y_val = train_test_split(processed_train, y_train, stratify = y_train, 
                                                  test_size = 0.2, random_state=42)

In [534]:
from sklearn import tree
from lightgbm import LGBMClassifier
my_classifier = LGBMClassifier(learning_rate=0.1)

In [535]:

processed_train

array([[0.  , 1.  , 0.1 , ..., 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.2 , ..., 0.  , 0.  , 0.  ],
       [1.  , 1.  , 0.2 , ..., 0.  , 0.  , 1.  ],
       ...,
       [0.  , 1.  , 0.2 , ..., 1.  , 0.  , 0.  ],
       [1.  , 1.  , 0.3 , ..., 0.  , 0.  , 1.  ],
       [0.  , 1.  , 0.45, ..., 0.  , 0.  , 0.  ]])

In [536]:
my_classifier.fit(X_Train,y_Train)
y_pred= my_classifier.predict(X_Val)
print(y_pred)

[1 0 0 ... 0 0 1]


In [538]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
 
print(1-accuracy_score(y_val, y_pred))

0.11094580233793838


In [539]:
test.bank_account = my_classifier.predict(processed_test)

In [540]:
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})

In [541]:
submission.sample(5)

,uniqueid,bank_account
5100,uniqueid_11235 x Rwanda,0
3272,uniqueid_9407 x Rwanda,0
8035,uniqueid_8310 x Tanzania,0
1968,uniqueid_8037 x Kenya,0
8070,uniqueid_8345 x Tanzania,0


In [542]:
submission.to_csv('nineth_submission.csv', index = False)
